## Weights Calculation
 Weight0: Every Gene weighted equally 
 
 Weight1: Ralitive Risk
 
 Weight2: Number of Hits
 
 Weight3: LofZ

In [ ]:
def calc_bias2(X,hw,cw,w_min=1e-6):
    m,W = _calc_W(X,hw,cw)
    W[W <= w_min] = 0

    M = np.dot(W,X)
    V = np.matmul(W,X**2) - \
        2*M*np.matmul(W,X) + \
        M**2
    Z = (X-M)/np.sqrt(V)

    return m,W,Z
def _calc_W(X,hw=1.0,wt=None):
    if wt is None:
        wt = np.repeat(1.,X.shape[1])
    m = np.average(X,axis=1,weights=wt)

    D = (m[np.newaxis,:] - m[:,np.newaxis])**2
    D[np.arange(X.shape[1]),
      np.arange(X.shape[1])] = np.inf
    v = (hw/(2 * np.sqrt(2*np.log(2))))**2

    W = np.exp(-D/(2*v))
    W = W / W.sum(axis=1)[:,np.newaxis]
    return m,W

# Load Files

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(1, '/home/jw3514/Work/ASD_Circuits/src')
from ASD_Circuits import *
from venn import venn

HGNC, ENSID2Entrez, GeneSymbol2Entrez, Entrez2Symbol = LoadGeneINFO()
ExpZ1Mat = pd.read_csv("../dat/allen-mouse-exp/AllenMouse_z1_mat.0511.csv", index_col=0)
ExpZ1Mat_QN = pd.read_csv("../dat/allen-mouse-exp/jw_z1_mat.qn.0418.csv", index_col=0)
g = LoadConnectome2()
ExpZ2Mat = pd.read_csv("../dat/allen-mouse-exp/AllenMouseBrain_Z2bias.csv", index_col=0)

# Spark

In [ ]:
Spark_Meta_2stage = pd.read_excel("../dat/genes/asd/TabS_DenovoWEST_Stage1+2.xlsx",
                           skiprows=2, sheet_name="TopDnEnrich")
Spark_Meta_ExomeWide = Spark_Meta_2stage[Spark_Meta_2stage["pDenovoWEST_Meta"]<=1.3e-6]
Spark_Meta_ExomeWide.shape

In [ ]:
Spark_Meta_ExomeWide[Spark_Meta_ExomeWide["HGNC"]=="LAMP5"]

In [ ]:
0.05/(1.3e-6)

In [ ]:
print("%.3e"%(0.05/19749))

In [ ]:
print("%.3e"%(0.05/18000))

In [ ]:
Agg_gene2None, Agg_gene2MutN = Aggregate_Gene_Weights2(Spark_Meta_ExomeWide, 
          allen_mouse_genes = ExpZ1Mat.index.values, out="../dat/Unionize_bias/Spark_Meta_EWS.GeneWeight.v2.csv")
len(Agg_gene2MutN)

In [ ]:
ASD_Z2 = AvgSTRZ_Weighted(ExpZ2Mat, Agg_gene2MutN, Method = 1,
                                        csv_fil = "../dat/Unionize_bias/Spark.EW.Z2.v2.csv")
D_Agg_avgZ_RD = RegionDistributions(ASD_Z2.set_index("STR"), show=True, topN=50)
ASD_Z2 = ASD_Z2.set_index("STR")

In [ ]:
GeneDF_top30 = Spark_Meta_2stage.head(30)
_, ASD_GeneW_top30 = Aggregate_Gene_Weights2(GeneDF_top30, 
          allen_mouse_genes = ExpZ2Mat.index.values, out="../dat/Unionize_bias/ASD.top30.gw.csv")
ASD_30 = AvgSTRZ_Weighted(ExpZ2Mat, ASD_GeneW_top30, Method = 1,
                                        csv_fil = "../dat/Unionize_bias/test.ASD.top30.Z2.csv")
D_Agg_avgZ_RD = RegionDistributions(ASD_30.set_index("STR"), show=True, topN=50)

In [ ]:
ASD_Z2_old = pd.read_csv("../dat/Unionize_bias/Spark_Meta_EWS.Z2.bias.csv", index_col="STR")

In [ ]:
#BiasCorrelation(ASD_Z2, ASD_Z2_old)

In [ ]:
old_weights = LoadGeneWeights("../dat/Unionize_bias/Spark_Meta_EWS.GeneWeight.csv")
ASD_Z2_old2 = AvgSTRZ_Weighted(ExpZ2Mat, old_weights, Method = 1)
ASD_Z2_old2 = ASD_Z2_old2.set_index("STR")

In [ ]:
BiasCorrelation(ASD_Z2, ASD_Z2_old)

In [ ]:
4 * 0.554 + 2*0.333

In [ ]:
ExpZ2Mat_bs = pd.read_csv("../dat/allen-mouse-exp/AllenMouseBrain_Z2bias.brainspanMatch.csv", index_col=0)

In [ ]:
ASD_Z2_bs = AvgSTRZ_Weighted(ExpZ2Mat_bs, Agg_gene2MutN, Method = 1,
                                        csv_fil = "ASD.EW.Z2.bs.csv")
D_Agg_avgZ_RD = RegionDistributions(ASD_Z2_bs.set_index("STR"), show=True, topN=50)

In [ ]:
ASD_Z2_bs.head(10)

In [ ]:
set(ASD_Z2.head(50)["STR"].values).difference(ASD_Z2_bs.head(50)["STR"].values)

In [ ]:
set(ASD_Z2_bs.head(50)["STR"].values).difference(ASD_Z2.head(50)["STR"].values)

In [ ]:
ASD_Z2_bs = ASD_Z2_bs.set_index("STR")

In [ ]:
ASD_Z2 = pd.read_csv("../dat/Unionize_bias/Spark_Meta_EWS.Z2.bias.csv", index_col="STR")
BiasCorrelation(ASD_Z2, ASD_Z2_bs)

In [ ]:
SCZ_Bias = pd.read_csv("../dat/Unionize_bias/test.SCZ.top61.Z2.csv")

In [ ]:
ExpZ2Mat_Res = pd.read_csv("Z2.Residue.csv", index_col=0)

In [ ]:
ASD_Z2_Res = AvgSTRZ_Weighted(ExpZ2Mat_Res, Agg_gene2MutN, Method = 1,
                                        csv_fil = "ASD.EW.Z12.Res.csv")
D_Agg_avgZ_RD = RegionDistributions(ASD_Z2_Res.set_index("STR"), show=True, topN=50)
ASD_Z2_Res = ASD_Z2_Res.set_index("STR")

In [ ]:
ASD_Z2_Res.head(50)

In [ ]:
BiasCorrelation(ASD_Z2, ASD_Z2_Res)

In [ ]:
ASD_Z1 = AvgSTRZ_Weighted(ExpZ1Mat, Agg_gene2MutN, Method = 1,
                                        csv_fil = "dat/bias2/New.ASD.EW.Z1.csv")
D_Agg_avgZ_RD = RegionDistributions(ASD_Z1.set_index("STR"), show=True, topN=50)

In [ ]:
STR_Exp_Mat = pd.read_csv("../dat/allen-mouse-exp/ExpLevel.csv", index_col=0)
ASD_Exp_lv = AvgSTRZ_Weighted(STR_Exp_Mat, Agg_gene2MutN, Method = 1,
                                        csv_fil = "../dat/Unionize_bias/ASD.EW.ExpL.csv")
D_Agg_avgZ_RD = RegionDistributions(ASD_Exp_lv.set_index("STR"), show=True, topN=50)

In [ ]:
ASD_Z1_qn = AvgSTRZ_Weighted(ExpZ1Mat_QN, Agg_gene2MutN, Method = 1, NonNeg=False, 
                                        csv_fil = "dat/bias2/New.ASD.EW.Z1.qn.csv")
D_Agg_avgZ_RD = RegionDistributions(ASD_Z1_qn.set_index("STR"), show=True, topN=50)

In [ ]:
ASD_Z1_nn = AvgSTRZ_Weighted(ExpZ1Mat, Agg_gene2MutN, Method = 1, NonNeg=True,
                                        csv_fil = "dat/bias2/New.ASD.EW.Z1.nn.csv")
ASD_Z1_qn_nn = AvgSTRZ_Weighted(ExpZ1Mat_QN, Agg_gene2MutN, Method = 1, NonNeg=True,
                                        csv_fil = "dat/bias2/New.ASD.EW.Z1.qn.nn.csv")

In [ ]:
D_Agg_avgZ_RD = RegionDistributions(ASD_Z1_nn.set_index("STR"), show=True, topN=50)

In [ ]:
D_Agg_avgZ_RD = RegionDistributions(ASD_Z1_qn_nn.set_index("STR"), show=True, topN=50)

In [ ]:
#ASD_Z1 = ASD_Z1.set_index("STR")
#ASD_Z1_qn = ASD_Z1_qn.set_index("STR")
dat1, dat2 = [], []
for STR in ASD_Z1.index.values:
    bias1 = ASD_Z1.loc[STR, "EFFECT"]
    bias2 = ASD_Z1_qn.loc[STR, "EFFECT"]
    dat1.append(bias1)
    dat2.append(bias2)
plt.scatter(dat1, dat2)
print(spearmanr(dat1, dat2))

In [ ]:
CompareList(ASD_Z1.head(50)["STR"].values, ASD_Z1_qn.head(50)["STR"].values)

# Z2 Calculation

In [ ]:
Match_DIR = "dat/gene_matches-remap_homolog"
Match_DIR = "../dat/genes/ExpMatch_RootExp_uniform_kernal/"
Agg_Match = MakeMatchDF(Agg_gene2MutN, N=1000, DIR=Match_DIR)
Agg_Match.to_csv("dat/bias2/New.Spark_Meta_EWS.Matches.csv")
Agg_Match.shape

In [ ]:
ASD_z2 = AvgSTRZ_Weighted(ExpZ1Mat, Agg_gene2MutN, Method = 2, Match_DF = Agg_Match,
                                csv_fil = "dat/bias2/New.ASD.EW.Z2.csv")
ASD_z2_qn = AvgSTRZ_Weighted(ExpZ1Mat_QN, Agg_gene2MutN, Method = 2, Match_DF = Agg_Match,
                                csv_fil = "dat/bias2/New.ASD.EW.Z2.qn.csv")

In [ ]:
RD_Agg_avgZ_RD = RegionDistributions(ASD_z2.set_index("STR"), show=True, topN=50)

In [ ]:
RD_Agg_avgZ_RD = RegionDistributions(ASD_z2_qn.set_index("STR"), show=True, topN=50)

In [ ]:
ASD_z2_nn = AvgSTRZ_Weighted(ExpZ1Mat, Agg_gene2MutN, Method = 2, Match_DF = Agg_Match, NonNeg=True,
                                csv_fil = "dat/bias2/New.ASD.EW.Z2.nn.csv")
ASD_z2_qn_nn = AvgSTRZ_Weighted(ExpZ1Mat_QN, Agg_gene2MutN, Method = 2, Match_DF = Agg_Match, NonNeg=True,
                                csv_fil = "dat/bias2/New.ASD.EW.Z2.qn.nn.csv")


In [ ]:
RD_Agg_avgZ_RD = RegionDistributions(ASD_z2_nn.set_index("STR"), show=True, topN=50)

In [ ]:
RD_Agg_avgZ_RD = RegionDistributions(ASD_z2_qn_nn.set_index("STR"), show=True, topN=50)

# Lastest ASD Mutations

In [ ]:
#SiblingDF = pd.read_csv("dat/ASD_Siblings.csv")
SiblingDF = pd.read_csv("dat/ASD_Sibling_LGD_Mis.2.0.csv")
Sibling_weights = Mut2GeneDF(SiblingDF, GeneSymbol2Entrez)
Sib_Z1 = AvgSTRZ_Weighted(ExpZ1Mat, Sibling_weights, Method = 1,
                    csv_fil = "dat/bias2/ASD.Siblings.z1.csv")
Sib_Z2 = AvgSTRZ_Weighted(ExpZ2Mat, Sibling_weights, Method = 1,
                    csv_fil = "dat/bias2/ASD.Siblings.z2.csv")
Sib_Z1_ng = AvgSTRZ_Weighted(ExpZ1Mat, Sibling_weights, Method = 1, NonNeg=True, 
                    csv_fil = "dat/bias2/ASD.Siblings.z1.ng.csv")
Sib_Z2_ng = AvgSTRZ_Weighted(ExpZ2Mat, Sibling_weights, Method = 1, NonNeg=True, 
                    csv_fil = "dat/bias2/ASD.Siblings.z2.ng.csv")

In [ ]:
Sib_Z1

In [ ]:
Spark_Meta = pd.read_excel("/Users/jiayao/Work/ASD_Circuits/dat/genes/asd/spark_Meta_top_genes.xlsx")
#Spark_Meta['pDenovoWEST_Meta'] = pd.to_numeric(Aggregate["pDenovoWEST_Meta"], errors='coerce')

In [ ]:
Agg_gene2None, Agg_gene2MutN = Aggregate_Gene_Weights2(Spark_Meta,  allen_mouse_genes, 
                out = "dat/bias2/meta.gene.weights.csv")
Agg_Match = MakeMatchDF(Agg_gene2MutN, N=1000)


In [ ]:
print(len(Agg_gene2MutN))

In [ ]:
Agg_Match.to_csv("dat/Final_Spark_Meta.Matches.csv")

In [ ]:
ASD_Meta_Spec = AvgSTRZ_Weighted(ExpZscoreMat, Agg_gene2MutN, Match_DF=Agg_Match, Method = 2,
                  NonNeg=True, csv_fil = "dat/bias2/ASD.Meta.159.NonNeg.Spec.bias.csv")
RD_Agg_avgZ_RD = RegionDistributions(ASD_Meta_Spec.set_index("STR"), show=True)

In [ ]:
ASD_Meta_Spec_Max0 = AvgSTRZ_Weighted(ExpZscoreMat, Agg_gene2MutN, Match_DF=Agg_Match, 
                    Method = 2, csv_fil = "dat/bias2/ASD.Meta.Spec.max0.bias.csv")
RD_Agg_avgZ_RD = RegionDistributions(ASD_Meta_Spec_Max0.set_index("STR"), show=True)

In [ ]:
ASD_E, ASD_P = CohesivenessProfile(ASD_Meta_Spec_Max0.set_index("STR"))
Sib_E, Sib_P = CohesivenessProfile(ASD_Meta_Spec_Max0.set_index("STR"))
%matplotlib inline
mpl.rcParams.update(mpl.rcParamsDefault)
fig, ax = plt.subplots(dpi=720, figsize=(16,8))
plt.style.use('seaborn-talk')
matplotlib.rcParams.update({'font.size': 32})

addline(topNs, Max0E, ax, "blue")

ax.hlines(xmin=min(topNs), xmax=max(topNs), y=1, ls="--", color="grey")
ax.grid(True)

ax.set_ylabel("Cohesiveness Effect Size",fontsize=28)
ax.set_xlabel("Number of Most Biased Structures",fontsize=28)
ax.tick_params(axis='x', labelsize=20)
ax.tick_params(axis='y', labelsize=20)

plt.tight_layout()
plt.show()

In [ ]:
ASD_Meta_Spec_norm = AvgSTRZ_Weighted(ExpZscoreMatNorm, Agg_gene2MutN, Match_DF=Agg_Match, Method = 2,
                                              csv_fil = "dat/bias2/ASD.Meta.Spec.norm.bias.csv")
RD_Agg_avgZ_RD = RegionDistributions(ASD_Meta_Spec_norm.set_index("STR"), show=True)


In [ ]:
ASD_Meta_Spec_norm2 = AvgSTRZ_Weighted(ExpZscoreMatNorm2, Agg_gene2MutN, Match_DF=Agg_Match, Method = 2,
                         csv_fil = "dat/bias2/ASD.Meta.Spec.norm2.bias.csv")
RD_Agg_avgZ_RD = RegionDistributions(ASD_Meta_Spec_norm2.set_index("STR"), show=True)

In [ ]:
ASD_Meta_Spec2 = Z2_GeneSet(ExpZscoreMat, Agg_gene2MutN, Match_DF=Agg_Match,
                                              csv_fil = "dat/bias2/ASD.Meta.2.Spec.bias.csv")
RD_Agg_avgZ_RD = RegionDistributions(ASD_Meta_Spec2.set_index("STR"), show=True)


In [ ]:
ASD_Meta_avgZ = AvgSTRZ_Weighted(ExpZscoreMat, Agg_gene2MutN, Match_DF=Agg_Match, Method = 1,
                                              csv_fil = "dat/bias2/ASD.Meta.avgZ.max0.bias.csv")

In [ ]:
RD_Agg_avgZ_RD = RegionDistributions(ASD_Meta_avgZ.set_index("STR"), show=True)

In [ ]:
ASD_Meta_avgZ = AvgSTRZ_Weighted(ExpZscoreMat, Agg_gene2MutN, Match_DF=Agg_Match, Method = 1,
                                              csv_fil = "dat/bias2/ASD.Meta.avgZ.bias.csv")
RD_Agg_avgZ_RD = RegionDistributions(ASD_Meta_avgZ.set_index("STR"), show=True)

In [ ]:
340/320

In [ ]:
new_gene_dict = {}
for k,v in Agg_gene2MutN.items():
    if k in nega_bias_strs:
        new_gene_dict[k] = v
new_gene_dict    

In [ ]:
ASD_Meta_Spec = AvgSTRZ_Weighted(ExpZscoreMat, new_gene_dict, Match_DF=Agg_Match, Method = 2,
                                              csv_fil = "dat/bias2/test.neg.ASD.Meta.Spec.bias.csv")
RD_Agg_avgZ_RD = RegionDistributions(ASD_Meta_Spec.set_index("STR"), show=True)

In [ ]:
ASD_Meta_Spec.head(50)

In [ ]:
ASD_Meta_Spec = pd.read_csv("dat/bias2/ASD.Meta.Spec.bias.csv", index_col="STR")

In [ ]:
ExpZscoreMat.loc[Agg_gene2MutN.keys(), "Anterior_pretectal_nucleus"].hist(bins=20)

In [ ]:
Z2_score = []
STR = "Anterior_pretectal_nucleus"
for entrez in Agg_gene2MutN.keys():
    xx = Z2(entrez, STR, ExpZscoreMat, Agg_Match)
    Z2_score.append(xx)
    if xx > 4:
        print(entrez, Entrez2Symbol[entrez])
plt.hist(Z2_score, bins=20)

In [ ]:
Z2_score = []
STR = "Mediodorsal_nucleus_of_thalamus"
for entrez in Agg_gene2MutN.keys():
    xx = Z2(entrez, STR, ExpZscoreMat, Agg_Match)
    Z2_score.append(xx)
    if xx > 4:
        print(entrez, Entrez2Symbol[entrez])
plt.hist(Z2_score, bins=20)

In [ ]:
ExpMat.loc[55023, SparkMetaBias.head(50).index.values]

In [ ]:
Z2_score = []
STR = "Nucleus_accumbens"
for entrez in Agg_gene2MutN.keys():
    xx = Z2(entrez, STR, ExpZscoreMat, Agg_Match)
    Z2_score.append(xx)
    if xx > 4:
        print(entrez, Entrez2Symbol[entrez])
plt.hist(Z2_score, bins=20)

In [ ]:
ExpZscoreMat.loc[Agg_gene2MutN.keys(), "Bed_nuclei_of_the_stria_terminalis"].hist(bins=20)

In [ ]:
ExpZscoreMat.loc[Agg_gene2MutN.keys(), "Ventral_tegmental_area"].hist()

In [ ]:
Z2_score = []
#STR = "Ventral_tegmental_area"
STR = "Prelimbic_area"
for entrez in Agg_gene2MutN.keys():
    xx = Z2(entrez, STR, ExpZscoreMat, Agg_Match)
    Z2_score.append(xx)
    if xx > 4:
        print(entrez, Entrez2Symbol[entrez])
plt.hist(Z2_score, bins=20)

In [ ]:
SparkMetaBias = pd.read_csv("dat/Jon_data/Spark_Final_ExpBias.csv")
SparkMetaBias.columns = ["STR", "EFFECT", "Rank", "NGene"]
SparkMetaBias = SparkMetaBias.set_index("STR")

In [ ]:
Box_dat_Zscores = []
Zscores_mean = []
for STR in SparkMetaBias.head(50).index.values:
    biases = ExpZscoreMat.loc[Agg_gene2MutN.keys(), STR].values
    Box_dat_Zscores.append(biases)
    Zscores_mean.append(np.mean(biases))

In [ ]:
Box_dat_Zscores_Z2 = []
scores_mean_Z2 = []
for STR in SparkMetaBias.head(50).index.values:
    Z2_score = []
    for entrez in Agg_gene2MutN.keys():
        xx = Z2(entrez, STR, ExpZscoreMat, Agg_Match)
        Z2_score.append(xx)
    Box_dat_Zscores_Z2.append(Z2_score)
    scores_mean_Z2.append(np.mean(Z2_score))

### Diff with Previous res

# Top60 De novo west (exome-wide-sig)

### Top X00 in Stage1

In [ ]:
Spark_Meta_1stage = pd.read_excel("~/Work/SPARK2020/TabS_DenovoWEST_Stage1.xlsx",
                           skiprows=1, sheet_name="AllGenes")
Spark_Meta_1stage.shape

In [ ]:
NGenes = 200
ASD_Meta_Muts_HC3 = ASD_Meta_Muts[ASD_Meta_Muts["HGNC"].isin(
    Spark_Meta_1stage["HGNC"].values[:NGenes])]
ASD_Meta_Muts_HC3.to_csv("dat/bias2/Spark_Meta_{}.Muts.csv".format(NGenes))
# Gene weights
_, Agg_gene2MutN3 = Aggregate_Gene_Weights2(Spark_Meta_1stage.head(NGenes), 
          allen_mouse_genes = allen_mouse_genes, 
          out = "dat/bias2/Spark_Meta_{}.GeneWeight.csv".format(NGenes))

Spark_Meta_ExomeWide.to_csv("dat/bias2/Spark_Meta_{}.Genes.csv".format(NGenes))

In [ ]:
NGenes = 800
ASD_Meta_Muts_HC3 = ASD_Meta_Muts[ASD_Meta_Muts["HGNC"].isin(
    Spark_Meta_1stage["HGNC"].values[:NGenes])]
ASD_Meta_Muts_HC3.to_csv("dat/bias2/Spark_Meta_{}.Muts.csv".format(NGenes))
# Gene weights
_, Agg_gene2MutN3 = Aggregate_Gene_Weights2(Spark_Meta_1stage.head(NGenes), 
          allen_mouse_genes = allen_mouse_genes, 
          out = "dat/bias2/Spark_Meta_{}.GeneWeight.csv".format(NGenes))
# Matches
Agg_Match = MakeMatchDF(Agg_gene2MutN3, N=1000)
Agg_Match.to_csv("dat/bias2/Spark_Meta_{}.Matches.csv".format(NGenes))
Spark_Meta_ExomeWide.to_csv("dat/bias2/Spark_Meta_{}.Genes.csv".format(NGenes))

In [ ]:
print(len(Agg_gene2MutN3))

In [ ]:
ASD_300_Z2 = AvgSTRZ_Weighted(ExpZ2Mat, Agg_gene2MutN3, Method = 1, NonNeg=False, 
                            csv_fil = "dat/bias2/ASD.{}.pLI.z2.csv".format(NGenes))
RD_Agg_avgZ_RD = RegionDistributions(ASD_300_Z2.set_index("STR"), show=True)

### Stage 2

In [ ]:
Spark_Meta_2stage = pd.read_excel("../dat/genes/asd/TabS_DenovoWEST_Stage1+2.xlsx",
                           skiprows=2, sheet_name="TopDnEnrich")
Spark_Meta_2stage.shape

In [ ]:
ASD_Meta_Muts_HC2 = ASD_Meta_Muts[ASD_Meta_Muts["HGNC"].isin(
    Spark_Meta_2stage["HGNC"].values)]
ASD_Meta_Muts_HC2.to_csv("dat/bias2/Spark_Meta_159.Muts.csv")
# Gene weights
Agg_gene2None, Agg_gene2MutN = Aggregate_Gene_Weights2(Spark_Meta_2stage, 
          allen_mouse_genes = allen_mouse_genes, 
          out = "dat/bias2/Spark_Meta_159.GeneWeight.csv")
# Matches
Agg_Match = MakeMatchDF(Agg_gene2MutN, N=1000)
Agg_Match.to_csv("dat/bias2/Spark_Meta_159.Matches.csv")
Spark_Meta_ExomeWide.to_csv("dat/bias2/Spark_Meta_159.Genes.csv")

In [ ]:
ASD_159_Z1_NoNeg = AvgSTRZ_Weighted(ExpZscoreMat, Agg_gene2MutN, Method = 1, NonNeg=True, 
                                              csv_fil = "dat/bias2/ASD.159.pLI.z1.NoNeg.csv")
RD_Agg_avgZ_RD = RegionDistributions(ASD_159_Z1_NoNeg.set_index("STR"), show=True)

In [ ]:
ASD_159_Z2 = AvgSTRZ_Weighted(ExpZ2Mat, Agg_gene2MutN, Method = 1, NonNeg=False, 
                                              csv_fil = "dat/bias2/ASD.159.pLI.z2.csv")
RD_Agg_avgZ_RD = RegionDistributions(ASD_159_Z2.set_index("STR"), show=True)

In [ ]:
ASD_Discov_Muts = pd.read_csv("../dat/genes/SPARK/ASD_Discov_DNVs.txt", delimiter="\t")
ASD_Replic_Muts = pd.read_csv("../dat/genes/SPARK/ASD_Rep_DNVs.txt", delimiter="\t")
ASD_Meta_Muts = pd.concat([ASD_Discov_Muts, ASD_Replic_Muts])
ASD_Meta_Muts_HC = ASD_Meta_Muts[ASD_Meta_Muts["HGNC"].isin(
    Spark_Meta_ExomeWide["HGNC"].values)]

In [ ]:
ASD_EW_Z2_NoNeg = AvgSTRZ_Weighted(ExpZ2Mat, Agg_gene2MutN, Method = 1, NonNeg=True, 
                                              csv_fil = "dat/bias2/ASD.EW.pLI.z2.NoNeg.csv")
RD_Agg_avgZ_RD = RegionDistributions(ASD_EW_Z2_NoNeg.set_index("STR"), show=True)

In [ ]:
Spark_Meta_ExomeWide = Spark_Meta_2stage[Spark_Meta_2stage["pDenovoWEST_Meta"]<=1.3e-6]
#Spark_Meta_ExomeWide = Spark_Meta_2stage[Spark_Meta_2stage[
#    "pDenovoWEST_Meta"]<=1e-2]
Spark_Meta_ExomeWide.shape

In [ ]:
Agg_gene2None, Agg_gene2MutN = Aggregate_Gene_Weights2(Spark_Meta_ExomeWide, allen_mouse_genes=ExpZ2Mat.index.values) 
          #out = "dat/bias2/Spark_Meta_EWS.GeneWeight.csv")
len(Agg_gene2MutN)

In [ ]:
# POGZ FOXP1 ANK2 SRCAP

In [ ]:
# Save Mutation / Match / Weight file for bootstrape/match use
Agg_Match = MakeMatchDF(Agg_gene2MutN, N=1000)
Agg_Match.to_csv("dat/bias2/Spark_Meta_EWS.Matches.csv")
Spark_Meta_ExomeWide.to_csv("dat/bias2/Spark_Meta_EWS.Genes.csv")

In [ ]:
ASD_Meta_Muts_HC.to_csv("dat/bias2/Spark_Meta_EWS.Muts.csv")

In [ ]:
ExpMat_Z1 = pd.read_csv("dat/ExpMat/Energy.ExpZ1.csv", index_col=0)
ExpMat_Z2 = pd.read_csv("dat/ExpMat/Energy.ExpZ2.csv", index_col=0)
ExpMat_Z1_QN = pd.read_csv("dat/ExpMat/Energy.ExpZ1.QN.csv", index_col=0)

In [ ]:
ASD_EW_exp = AvgSTRZ_Weighted(ExpMat, Agg_gene2MutN, Method = 1,
                                              csv_fil = "dat/bias2/ASD.EW.pLI.expL.csv")
RD_Agg_avgZ_RD = RegionDistributions(ASD_EW_exp.set_index("STR"), show=True)

In [ ]:
EWE, EWP = CohesivenessProfile(ASD_EW_exp.set_index("STR"), topNs=topNs, g=g)
PlotCohesivenessProfile(topNs, EWE)

In [ ]:
ASD_EW_Z1 = AvgSTRZ_Weighted(ExpMat_Z1, Agg_gene2MutN, Method = 1,
                                              csv_fil = "dat/bias2/ASD.EW.pLI.z1.csv")
RD_Agg_avgZ_RD = RegionDistributions(ASD_EW_Z1.set_index("STR"), show=True)

In [ ]:
ASD_EW_Z2 = AvgSTRZ_Weighted(ExpMat_Z2, Agg_gene2MutN, Method = 1,
                                              csv_fil = "dat/bias2/ASD.EW.pLI.z2.csv")
RD_Agg_avgZ_RD = RegionDistributions(ASD_EW_Z2.set_index("STR"), show=True)

In [ ]:
ASD_EW_Z1_QN = AvgSTRZ_Weighted(ExpMat_Z1_QN, Agg_gene2MutN, Method = 1,
                                              csv_fil = "dat/bias2/ASD.EW.pLI.z1.qn.csv")
RD_Agg_avgZ_RD = RegionDistributions(ASD_EW_Z1_QN.set_index("STR"), show=True)

In [ ]:
ASD_EW_Z1_NG = AvgSTRZ_Weighted(ExpMat_Z1, Agg_gene2MutN, Method = 1, NonNeg=True,
                                              csv_fil = "dat/bias2/ASD.EW.pLI.z1.noneg.csv")
RD_Agg_avgZ_RD = RegionDistributions(ASD_EW_Z1_NG.set_index("STR"), show=True)

In [ ]:
ASD_EW_Z2_NG = AvgSTRZ_Weighted(ExpMat_Z2, Agg_gene2MutN, Method = 1, NonNeg=True,
                                              csv_fil = "dat/bias2/ASD.EW.pLI.z2.noneg.csv")
RD_Agg_avgZ_RD = RegionDistributions(ASD_EW_Z2_NG.set_index("STR"), show=True)

In [ ]:
ASD_EW_Z1_QN_NG = AvgSTRZ_Weighted(ExpMat_Z1_QN, Agg_gene2MutN, Method = 1, NonNeg=True,
                                              csv_fil = "dat/bias2/ASD.EW.pLI.z1.qn.ng.csv")
RD_Agg_avgZ_RD = RegionDistributions(ASD_EW_Z1_QN_NG.set_index("STR"), show=True)

In [ ]:
ASD_EW_Z1 = AvgSTRZ_Weighted(ExpZscoreMat, Agg_gene2MutN2, Method = 1,
                                              csv_fil = "dat/bias2/ASD.EW.pLI.z1.2.csv")
RD_Agg_avgZ_RD = RegionDistributions(ASD_EW_Z1.set_index("STR"), show=True)

In [ ]:
ASD_EW_Z2 = AvgSTRZ_Weighted(ExpZ2Mat, Agg_gene2MutN, Method = 1,
                                              csv_fil = "dat/bias2/ASD.EW.pLI.z2.csv")
RD_Agg_avgZ_RD = RegionDistributions(ASD_EW_Z2.set_index("STR"), show=True)

In [ ]:
ZscoreMat = pd.read_csv("dat/energy-exp_spec.csv", index_col="ROW")
ASD_EW_Z1_NoNeg = AvgSTRZ_Weighted(ZscoreMat, Agg_gene2MutN2, Method = 1, NonNeg=True, 
                                              csv_fil = "dat/bias2/ASD.EW.pLI.z1.NoNeg.2.csv")
RD_Agg_avgZ_RD = RegionDistributions(ASD_EW_Z1_NoNeg.set_index("STR"), show=True)

In [ ]:
#Exp
ASD_EW_Z2_NoNeg = AvgSTRZ_Weighted(ExpZ2Mat, Agg_gene2MutN, Method = 1, NonNeg=True, 
                                              csv_fil = "dat/bias2/ASD.EW.pLI.z2.NoNeg.csv")
RD_Agg_avgZ_RD = RegionDistributions(ASD_EW_Z2_NoNeg.set_index("STR"), show=True)

In [ ]:
topNs = list(range(200, 60, -5)) + list(range(60, 40, -1)) + list(range(40, 10, -5))

In [ ]:
np.arange(200, 10, -5)

In [ ]:
g2 = LoadConnectome2("../dat/allen-mouse-conn/adj_mat_cartesian_distal.csv")

In [ ]:
EWE, EWP = CohesivenessProfile(ASD_EW_Z1.set_index("STR"), topNs=topNs, g=g)
EWE2, EWP2 = CohesivenessProfile(Sib_Z1.set_index("STR"), topNs=topNs, g=g)
PlotCohesivenessProfile(topNs, EWE, EWE2)

In [ ]:
EWE, EWP = CohesivenessProfile(ASD_EW_Z1.set_index("STR"), topNs=topNs, g=g2)
EWE2, EWP2 = CohesivenessProfile(Sib_Z1.set_index("STR"), topNs=topNs, g=g2)
PlotCohesivenessProfile(topNs, EWE, EWE2)

In [ ]:
EWE, EWP = CohesivenessProfile(ASD_EW_Z2.set_index("STR"), topNs=topNs, g=g)
EWE2, EWP2 = CohesivenessProfile(Sib_Z2.set_index("STR"), topNs=topNs, g=g)
PlotCohesivenessProfile(topNs, EWE, EWE2)

In [ ]:
EWE, EWP = CohesivenessProfile(ASD_EW_Z1_NoNeg.set_index("STR"), topNs=topNs, g=g)
EWE2, EWP2 = CohesivenessProfile(Sib_Z1_ng.set_index("STR"), topNs=topNs, g=g)
PlotCohesivenessProfile(topNs, EWE, EWE2)

In [ ]:
EWE, EWP = CohesivenessProfile(ASD_EW_Z1_NoNeg.set_index("STR"), topNs=topNs, g=g2)
EWE2, EWP2 = CohesivenessProfile(Sib_Z1_ng.set_index("STR"), topNs=topNs, g=g2)
PlotCohesivenessProfile(topNs, EWE, EWE2)

In [ ]:
EWE, EWP = CohesivenessProfile(ASD_EW_Z2_NoNeg.set_index("STR"), topNs=topNs, g=g)
EWE2, EWP2 = CohesivenessProfile(Sib_Z2_ng.set_index("STR"), topNs=topNs, g=g)
PlotCohesivenessProfile(topNs, EWE, EWE2)

In [ ]:
EWE, EWP = CohesivenessProfile(ASD_EW_Z2_NoNeg.set_index("STR"), topNs=topNs, g=g2)
EWE2, EWP2 = CohesivenessProfile(Sib_Z2_ng.set_index("STR"), topNs=topNs, g=g2)
PlotCohesivenessProfile(topNs, EWE, EWE2)

In [ ]:
np.array(EWP2)<0.05

In [ ]:
PlotCohesivenessProfile(topNs, EWE, EWE2)

In [ ]:
def NanToV(value, V):
    if value != value:
        return V
    else:
        return value
def QuantileNorm(Values):
    minV = min(Values)
    Values_tosort = [NanToV(x, minV-1) for x in Values]
    Total = len([x for x in Values if x==x])
    res = []
    idx = np.argsort(Values_tosort)
    for i, v in zip(idx, Values):
        if v==v:
            quantile = i/Total
            res.append(quantile)
        else:
            res.append(0)
    return np.array(res)

In [ ]:
dat = []
for col in ExpZscoreMat.columns.values[:]:
    #print(col)
    Values = ExpZscoreMat[col].values
    quantiles = QuantileNorm(Values)
    dat.append(quantiles)
MouseZQ = pd.DataFrame(data=dat, index = ExpZscoreMat.columns.values[:],
                          columns = ExpZscoreMat.index.values)
MouseZQ = MouseZQ.transpose()

In [ ]:
dat = []
for g, row in ExpMat.iterrows():
    #print(col)
    Values = row.values
    quantiles = QuantileNorm(Values)
    dat.append(quantiles)
MouseQ = pd.DataFrame(data=dat, index = ExpMat.index.values[:],
                          columns = ExpMat.columns.values)

In [ ]:
ASD_EW_Q = AvgSTRZ_Weighted(MouseQ, Agg_gene2MutN, Method = 1,
                                              csv_fil = "dat/bias2/ASD.EW.pLI.q.csv")
RD_Agg_avgZ_RD = RegionDistributions(ASD_EW_Q.set_index("STR"), show=True)

In [ ]:
ASD_EW_Q = AvgSTRZ_Weighted(MouseZQ, Agg_gene2MutN, Method = 1,
                                              csv_fil = "dat/bias2/ASD.EW.pLI.zq.csv")
RD_Agg_avgZ_RD = RegionDistributions(ASD_EW_Q.set_index("STR"), show=True)

In [ ]:
EWE, EWP = CohesivenessProfile(ASD_EW_Q.set_index("STR"), g=g)

In [ ]:
%matplotlib inline
mpl.rcParams.update(mpl.rcParamsDefault)
fig, ax = plt.subplots(dpi=720, figsize=(16,8))
plt.style.use('seaborn-talk')
matplotlib.rcParams.update({'font.size': 32})

addline(topNs, EWE, ax, "blue")

ax.hlines(xmin=min(topNs), xmax=max(topNs), y=1, ls="--", color="grey")
ax.grid(True)

ax.set_ylabel("Cohesiveness Effect Size",fontsize=28)
ax.set_xlabel("Number of Most Biased Structures",fontsize=28)
ax.tick_params(axis='x', labelsize=20)
ax.tick_params(axis='y', labelsize=20)

plt.tight_layout()
plt.show()

In [ ]:
BiasCorrelation(ASD_EW_Z1.set_index("STR"), ASD_EW_Z2.set_index("STR"))

In [ ]:
BiasCorrelation(ASD_EW_Z1_No.set_index("STR"), ASD_EW_Z2.set_index("STR"))

In [ ]:
ASD_EW_Spec = AvgSTRZ_Weighted(ExpZscoreMat, Agg_gene2MutN, Match_DF=Agg_Match, Method = 2,
                                              csv_fil = "dat/bias2/ASD.EW.pLI.Spec.bias.csv")
RD_Agg_avgZ_RD = RegionDistributions(ASD_EW_Spec.set_index("STR"), show=True)

In [ ]:
EWE, EWP = CohesivenessProfile(ASD_EW_Spec.set_index("STR"))

In [ ]:
%matplotlib inline
mpl.rcParams.update(mpl.rcParamsDefault)
fig, ax = plt.subplots(dpi=720, figsize=(16,8))
plt.style.use('seaborn-talk')
matplotlib.rcParams.update({'font.size': 32})

addline(topNs, EWE, ax, "blue")

ax.hlines(xmin=min(topNs), xmax=max(topNs), y=1, ls="--", color="grey")
ax.grid(True)

ax.set_ylabel("Cohesiveness Effect Size",fontsize=28)
ax.set_xlabel("Number of Most Biased Structures",fontsize=28)
ax.tick_params(axis='x', labelsize=20)
ax.tick_params(axis='y', labelsize=20)

plt.tight_layout()
plt.show()

In [ ]:
Jon_60 = pd.read_csv("dat/jon_data/exprank-for_jw.csv", index_col="STR")
EWE, EWP = CohesivenessProfile(Jon_60, g=g)

In [ ]:
%matplotlib inline
mpl.rcParams.update(mpl.rcParamsDefault)
fig, ax = plt.subplots(dpi=720, figsize=(16,8))
plt.style.use('seaborn-talk')
matplotlib.rcParams.update({'font.size': 32})

addline(topNs, EWE, ax, "blue")

ax.hlines(xmin=min(topNs), xmax=max(topNs), y=1, ls="--", color="grey")
ax.grid(True)

ax.set_ylabel("Cohesiveness Effect Size",fontsize=28)
ax.set_xlabel("Number of Most Biased Structures",fontsize=28)
ax.tick_params(axis='x', labelsize=20)
ax.tick_params(axis='y', labelsize=20)

plt.tight_layout()
plt.show()

In [ ]:
Agg_gene2None, Agg_gene2MutN = Aggregate_Gene_Weights2(Spark_Meta_ExomeWide,
        allen_mouse_genes = allen_mouse_genes, 
          out = "dat/bias2/meta.ew.gene.weights.csv")
ASD_EW_Spec = AvgSTRZ_Weighted(ExpZscoreMat, Agg_gene2MutN, Match_DF=Agg_Match, Method = 2, 
                           NonNeg=True, csv_fil = "dat/bias2/ASD.EW.pLI.NonNeg.Spec.bias.csv")
RD_Agg_avgZ_RD = RegionDistributions(ASD_EW_Spec.set_index("STR"), show=True)

In [ ]:
EWE, EWP = CohesivenessProfile(ASD_EW_Spec.set_index("STR"), g=g)
mpl.rcParams.update(mpl.rcParamsDefault)
fig, ax = plt.subplots(dpi=720, figsize=(16,8))
plt.style.use('seaborn-talk')
matplotlib.rcParams.update({'font.size': 32})

addline(topNs, EWE, ax, "blue")

ax.hlines(xmin=min(topNs), xmax=max(topNs), y=1, ls="--", color="grey")
ax.grid(True)

ax.set_ylabel("Cohesiveness Effect Size",fontsize=28)
ax.set_xlabel("Number of Most Biased Structures",fontsize=28)
ax.tick_params(axis='x', labelsize=20)
ax.tick_params(axis='y', labelsize=20)

plt.tight_layout()
plt.show()

In [ ]:
Agg_gene2None, Agg_gene2MutN = Aggregate_Gene_Weights2(Spark_Meta_ExomeWide, pLI=False,
         allen_mouse_genes = allen_mouse_genes,  out = "dat/bias2/meta.ew.gene.weights.csv")
ASD_EW_Spec = AvgSTRZ_Weighted(ExpZscoreMat, Agg_gene2MutN, Match_DF=Agg_Match, Method = 2, 
                           NonNeg=True, csv_fil = "dat/bias2/ASD.EW.NonNeg.Spec.bias.csv")
RD_Agg_avgZ_RD = RegionDistributions(ASD_EW_Spec.set_index("STR"), show=True)

In [ ]:
Agg_gene2None, Agg_gene2MutN = Aggregate_Gene_Weights2(Spark_Meta_ExomeWide, pLI=True,
         allen_mouse_genes = allen_mouse_genes,  out = "dat/bias2/meta.ew.gene.weights.csv")
ASD_EW_AvgZ = AvgSTRZ_Weighted(ExpZscoreMat, Agg_gene2MutN, Match_DF=Agg_Match, Method = 1, 
                           NonNeg=True, csv_fil = "dat/bias2/ASD.EW.NonNeg.avgZ.bias.csv")
RD_Agg_avgZ_RD = RegionDistributions(ASD_EW_AvgZ.set_index("STR"), show=True)

# ASC

In [ ]:
asc_df = pd.read_csv("../dat/genes/asd/asc_tada_RR.csv")
#asd_asc_match_df = pd.read_csv("dat/matching/asd_asc_exp_matches_1000.new.csv", index_col="GENE")
#asc_gene2MutN = ASC_Gene_Weights(asc_df, gnomad_cons, FDR=0.1)

In [ ]:
asc_df.columns.values

In [ ]:
asc_df102 = asc_df[asc_df["qval_dnccPTV"]<=0.1]

In [ ]:
Asc_gene2MutN = ASC_Gene_Weights(asc_df102)
#Asc_Match = MakeMatchDF(Asc_gene2MutN, N=1000)

In [ ]:
#Asc_Match.to_csv("dat/ASC102.Matches.csv")

In [ ]:
ASC_102_bias = AvgSTRZ_Weighted(ExpZ2Mat, Asc_gene2MutN, 
                        Method = 1, csv_fil = "../dat/Unionize_bias/ASD.ASC102.Z2.bias.csv")
ASC_102_bias = ASC_102_bias.set_index("STR")
RD_Agg_avgZ_RD = RegionDistributions(ASC_102_bias, show=True)

In [ ]:
ASC_102_bias.head(20)

In [ ]:
BiasCorrelation(ASD_Z2, ASC_102_bias, compare=50)

In [ ]:
ScoreMatDir="/home/jw3514/Work/ASD_Circuits/dat/allen-mouse-conn/ScoreingMat_jw_v3/"
InfoMat=pd.read_csv(ScoreMatDir + "InfoMat.Ipsi.csv", index_col=0)
topNs = list(range(200, 5, -1))
STR_Ranks = ASC_102_bias.index.values
asd_topN_score = []
for topN in topNs:
    top_strs = STR_Ranks[:topN]
    asd_score = ScoreCircuit_SI_Joint(top_strs, InfoMat)
    asd_topN_score.append(asd_score)
asd_topN_score = np.array(asd_topN_score)

In [ ]:
import matplotlib.backends.backend_pdf

#fig, (ax1, ax2, ax3) = plt.subplots(3,1, dpi=720, figsize=(8,12))
fig, (ax1) = plt.subplots(1,1, dpi=120, figsize=(8,4))
BarLen = 34.1
#BarLen = 47.5
ax1.plot(topNs, asd_topN_score, color="blue", marker="o", markersize=5, lw=1,
                     ls="dashed", label="ASC 102")

ax1.set_xlabel("Structure Rank")
ax1.set_ylabel("Circuit Score")
plt.grid(True)
plt.tight_layout()
plt.legend()